<h1 align=center><font size = 5> Titanic:only use two features to reach top 5%</font></h1>

## Table of Contents

#### Qian Liu
- email:liuqiantx1221@163.com
- <18/07/2019>

<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Understand the data](#0)<br>
[Import data set](#1)<br>
[Understand the features](#2)<br>
2. [Feature engineering](#3)<br>
[Add new features](#4)<br>
[Filling missing values](#5)<br>
[Modify error information](#6)<br>
3. [Analysis of relationship between new features and results](#7)<br>
[An overview of features and Survived](#8)<br>
[Check the survival rate of all types of passengers](#9)<br>
[Survival prediction](#10)<br>
4. ["ship team" and "secondary family"](#11)<br>
[Passengers with immediate family members](#12)<br>
[Identify the "ship team"](#13)<br>
[Add new feature “If_Survived”](#14)<br>
[An overview of "If_Survived" and "Survived"](#15)<br>
[Survival prediction](#16)<br>
5. [Analysis of the remaining datasets](#17)<br>
[Learn about the remaining training data and test data](#18)<br>
[Visualization of the relationship between passenger's features and survival rate](#19)<br>
[Survival prediction and result submission](#20)<br>

## Introduction
This is my first kernel at Kaggle，I have learned a lot in Kaggle these two months,and I'm glad to share my kernel with you.Actually,I'm still a beginner in data science,so,if you have any suggestions, please leave a message or email me.:)

This kernel only uses knowledge of feature engineering, but for the use of pandas, I think this will be a good review.The core of the method is to decompose layer by layer, looking for “ship team” and “secondary family”.The analysis logic is somewhat complicated, but each step has annotations, and some also have charts. If there are any unclear points, please leave a message.I hope this kernel can help you.

# 1.Understand the data<a id="0"></a>

## 1.1 Import data set<a id="1"></a>

In [1]:
import pandas as pd
import numpy as np
import re
from numpy import nan
import warnings
warnings.filterwarnings("ignore")

In [1]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [1]:
train.sample(2)

In [1]:
test.sample(2)

In [1]:
#combine data
combine = pd.concat([train,test],sort = True,ignore_index=True)  

## 1.2 Understand the features<a id="2"></a>

In [1]:
combine.info()

### 1.2.1 The analysis of data category and meaning

#### A.Category data
**1.Nominal variable**
- "Cabin":The combination of letters and numbers,it indicates the passenger compartment and it has a great amount of missing data.
- "Embarked"：S,Q,C，it represent different ports.

**2.Binary variable**
- "Sex":male,female

**3.Ordinal variables**
- "Pclass ":1（high）,2（medium）,3（low），it represents socioeconomic status

#### B.Numerical data
**1.Discrete variable**
- "PassengerId"，Passenger's unique identifier
- "SibSp"：The number of siblings and spouses on board.
- "Parch"：The number of children and parents on board,including stepparents.
- "Survived"：Target variable.

**2.Continuous variable**
- "Age"
- "Fare"

#### C.Text variable
- "Name"：It is composed of family name, name, title, maiden name, nickname, etc. It maybe the most important feature of all, So I will conduct an in-depth analysis later.
- "Ticket"：There are many people in the same ticket number.they may be from the same family，but it not sure

### 1.2.2 Analysis of the relationship between independent variables and dependent variables

In [1]:
%%HTML
<div class='tableauPlaceholder' id='viz1563348721864' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ov&#47;OverviewofWomanData&#47;OverviewofWomanData&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='OverviewofWomanData&#47;OverviewofWomanData' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ov&#47;OverviewofWomanData&#47;OverviewofWomanData&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1563348721864');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1024px';vizElement.style.height='694px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

**Observation conclusion：**
- 1.The average survival rate for female is 74% when considering the effects of all variables
- 2."Pclass" has the greatest impact on survival rate，Especially for P1 and P2 female, their average survival rate is 95%.It can be understood that the survival rate of upper-class passengers is higher than that of the lower class.
- 3."Embarked" also has some impact on survival rate,as the survival rate of C port is 88%,higher than average survival,and according to 【Embarked,Pclass】，When Embarked is not considered, the survival rate of P3 is 50%. When Embarked is considered, the survival rate of each port changes greatly.
- 4."Age" is normal distribution，its impact on survival rate is not obvious.
- 5.The distribution of "Fare" has a very large positive skew.For most passenger,its impact on survival rate is not obvious,But overall, passengers with high fares also have higher survival rates.According to 【Fare,Pclass】，except Pclass 1,the other's distribution is concentrated.
- 6.The distribution shape and impact of SibSp and Parch are familiar,and they both represent family relationships,so they can be combined.

In [1]:
%%HTML
<div class='tableauPlaceholder' id='viz1563351259461' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ov&#47;OverviewofManData&#47;EmbarkedandPclass&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='OverviewofManData&#47;EmbarkedandPclass' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ov&#47;OverviewofManData&#47;EmbarkedandPclass&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1563351259461');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

**Observation conclusion：**
- 1.The average survival rate for male is 19% which is much lower than female.
- 2."Pclass" also has some impact on male survival rate，for example, the survival rate for P1 male is 37%,but according to the probability, none of them can predict to survive.
- 3.For male,Embarked C has the highest survival rate,but only 30%.Embarked Q survival rate is below average, only 7%.According to 【Embarked and Pclass】，since the Q port is almost all P3 male, and there are many P1 male embarked from C port,the impact of "Embarked" on male survival is not clear.
- 4.The survival rate of young men is much higher than the average, reaching 65%.According to 【Pclass and Age】，the survival rate of P1 and P2 young men has reached 100%, which is also in line with the “principle of women and children”.
- 5.The distribution and impact of "Fare" in male data is familiar with it in female data.

# 2.Feature engineering<a id="3"></a>

## 2.1 Add new features<a id="4"></a>

### 1A."Surname"
- Extract family names to identify passengers belonging to the same family

- "Name" includes family name, title, name, nickname(partial), and maiden name(partial),the following focuses on family name, title, and maiden name.

In [1]:
combine[["Name"]].sample(10)

In [1]:
combine["Surname"] = combine["Name"].str.split(",",expand = True)[0]
combine["Surname"] = combine["Surname"].str.split("-",expand = True)[0]
combine[["Name","Surname"]].head(2)

### 1B."Surname_Counts"

In [1]:
#Number of passengers with the same family name
for name in combine.Surname.values:
    combine.loc[combine.Surname==name,"Surname_Counts"]=combine[combine.Surname==name].shape[0]
combine[["Surname","Surname_Counts"]].head(2)

### 2."Gender"

In [1]:
combine['Title'] = combine.Name.str.extract('([A-Za-z]+)\.', expand=False)

In [1]:
#Distinguish passenger titles by gender
for i,j in combine.groupby(["Sex"]):
    j_title_list = set(list(j.Title.values))
    print(i,j_title_list)

- In addition to the common titles such as Ms., Mr. and Miss, it is observed that the male contains "Master", which means "boy". According to previous observations, the survival rate of young men is higher, so "Mater" can be extracted. ,  while other titles are still used as a sign to distinguish between men and women.

In [1]:
female_list = ['Lady', 'Dona', 'Ms', 'Countess', 'Mrs', 'Mme', 'Miss', 'Dr', 'Mlle']
male_list = ['Don', 'Sir',  'Capt', 'Rev', 'Jonkheer', 'Col', 'Mr', 'Major', 'Dr']
boy_list = ['Master']
combine.loc[(combine.Title.isin(female_list))&(combine.Sex=="female"),"Gender"]="Woman"
combine.loc[(combine.Title.isin(male_list))&(combine.Sex=="male"),"Gender"]="Man"
combine.loc[(combine.Title.isin(boy_list)),"Gender"]="Boy"

- In addition to "Title", "Age" is another way to distinguish between boys and men,according to the visual chart of【Pclass and Age】，male under 16 years old is also "Boy",what's more,we can also combine the two methods.

In [1]:
type1 = combine[(combine.Gender=="Boy")].shape[0]
type2 = combine[(combine.Sex=="male")&(combine.Age<16)].shape[0]
type3 = combine[((combine.Sex=="male")&(combine.Age<16))|((combine.Gender=="Boy"))].shape[0]
print("The amount of Boy in method1 is %d and the amount of Boy in method2 is %d and the amount of Boy in method2 is %d"%(type1,type2,type3))

In [1]:
survival_1 = combine[(combine.Gender=="Boy")&(combine.Pclass.isin([1,2]))]["Survived"].mean()
survival_2 = combine[(combine.Sex=="male")&(combine.Age<16)&(combine.Pclass.isin([1,2]))]["Survived"].mean()
survival_3 = combine[(((combine.Sex=="male")&(combine.Age<16))|(combine.Gender=="Boy"))&(combine.Pclass.isin([1,2]))]["Survived"].mean()
print("The Survival probability of method1 is %.2f%%,the survival probability of method2 is %.2f%%,and the survival probability of method3 is %.2f%%"
      %(survival_1*100,survival_2*100,survival_3*100))

- In the three ways, the survival rate of "P1 and P2 Boys" is 100%, but the number of boys in method 3 is the largest. Therefore,method 3 is used as the way to distinguish between boys and men.

In [1]:
combine.loc[(combine.Sex=="male")&(combine.Age<16),"Gender"]="Boy"

In [1]:
combine[combine.Gender=="Boy"][["Gender","Sex","Title","Age"]].sample(3)

- Since "Gender", "Title" and "Sex" are repeated, "Title" and "Sex" can be removed.

In [1]:
combine.drop(["Title","Sex"],axis = 1,inplace=True)

### 3A."Maiden"
- Extract the maiden name as another sign to identify family members

In [1]:
combine_female = combine[combine.Gender == "Woman"]
combine_female.reset_index(inplace = True)

for i,name in enumerate(list(combine_female.Name.values)):
    name = str(name)
    name = re.sub("\\s$","",name)
    name = re.sub(".*[^\\)]$","",name)
    name = re.sub(".*\\s(.*)\\)$","\\1",name)
    name = re.sub("^\\(","",name)
    combine_female.loc[i,"Maiden_name"] = name
combine_female[["Name","Maiden_name","Age"]].head(3)

In [1]:
#Add maiden name to the full dataset
for id in combine_female.PassengerId.values:
    id2 = int(id)
    name = combine_female[combine_female.PassengerId==id]["Maiden_name"].values
    combine.loc[combine.PassengerId==id2,"Maiden"] = name
combine[["Name","Maiden"]].head(2)

In [1]:
combine.loc[combine.Maiden=="","Maiden"]=nan

### 3B."Maiden_Counts"
- When the maiden name is repeated, there may be relatives on board.

In [1]:
for mai in combine.Maiden.values:
    combine.loc[combine.Maiden==mai,"Maiden_Counts"]=combine[combine.Maiden==mai].shape[0]
combine[["Maiden","Maiden_Counts"]].head(2)

### 4."Family_size"
- The "Family_size" here only represents a first-class family consisting of immediate family members.(Note:Due to the existence of a relationship such as step-parent, family members in the same family may have different surnames.)

In [1]:
combine["Family_size"] = combine["Parch"]+combine["SibSp"]+1
combine[["Name","Surname_Counts","Maiden_Counts","Family_size"]].head(2)

### 5."Pclass_Gender"

In [1]:
pclass_gender_list = []
for pclass,gender in zip(combine.Pclass.values,combine.Gender.values):
    pclass_gender = "P"+str(pclass)+"-"+str(gender)
    pclass_gender_list = pclass_gender_list+[pclass_gender]

In [1]:
combine["Pclass_Gender"] = pclass_gender_list
combine[["Pclass","Gender","Pclass_Gender"]].head(2)

### 6A."Ticket"(New)

In [1]:
combine[["Ticket"]].sample(5)

In [1]:
combine["Ticket"].value_counts()[:5]

- Passengers under the same ticket number as "members of the same ship group", the survival rate between "members of the same ship group" may affect each other.

In [1]:
#Extract the numeric part of the ticket number as the new ticket number
combine["Ticket_new"] = combine["Ticket"].str.extract('([0-9]+)$',expand = False)

In [1]:
combine[combine.Ticket_new.isna()][["Ticket","Ticket_new"]]

In [1]:
#Modify the ticket number without numeric part to the original ticket number
combine.loc[combine.Ticket_new.isna(),"Ticket_new"]="LINE"

In [1]:
#Replace the original ticket number with the new ticket number
for i,tic in enumerate(combine.Ticket_new.values):
    combine.loc[i,"Ticket"] = tic
combine.drop(["Ticket_new"],axis = 1,inplace = True)

In [1]:
combine["Ticket"].value_counts()[:5]

- If the “number of members of the same ship team” is inconsistent with the “Family_size”, the passenger may still have immediate family members in other “ship team”. It can form a "secondary family."

### 6B.“Ticket_Counts”

In [1]:
tic_list = combine.Ticket.values
for i in tic_list:
    combine.loc[combine["Ticket"]==i,"Ticket_Counts"] = combine[combine["Ticket"] == i].shape[0]
combine[["Ticket","Ticket_Counts"]].head(2)

### 7A."Ticket2"
- Some family members may be present in the adjacent ticket number, which can also be considered as "the same ship team"。（This rule is effective only when there is a kinship relationship.）

In [1]:
for i,ti in enumerate(combine.Ticket.values):
    t = list(ti)[:-2]+list("xx")
    d = ''
    combine.loc[i,"Ticket2"] = d.join(t)

### 7B."Ticket2_Counts"

In [1]:
tic2_list = combine.Ticket2.values
for i in tic2_list:
    combine.loc[combine["Ticket2"]==i,"Ticket2_Counts"] = combine[combine["Ticket2"] == i].shape[0]
combine[["Ticket","Ticket2","Ticket2_Counts"]].head(2)

### 8."Cabin_Title"
- Although there are many missing values in "Cabin", we can still extract the effective information from it for backup.

In [1]:
combine[combine.Cabin.notna()][["Cabin"]].sample(5)

- It seems that the initials of the cabin are more useful, but some of the cabin values contain multiple cabin, so the initials of the single cabin and the initials of the multi-cabin can be extracted separately.

In [1]:
have_cabin = combine[combine.Cabin.notna()]
have_cabin.reset_index(inplace = True)
multi_cabin = []
single_cabin = []
for i,j in enumerate(have_cabin.Cabin.values):
    passid = have_cabin.loc[i].PassengerId
    if len(j)>4:
        multi_cabin = multi_cabin+[passid]
    else:
        single_cabin = single_cabin+[passid]
print(multi_cabin)

In [1]:
#Extract the initials of a single cabin
cabin_title = have_cabin[have_cabin.PassengerId.isin(single_cabin)].Cabin.str.extract('^([A-Z])',expand=False)
have_cabin.loc[have_cabin.PassengerId.isin(single_cabin),"Cabin_Title"] = cabin_title
have_cabin[["Cabin","Cabin_Title"]].head(2)

In [1]:
have_cabin[have_cabin.PassengerId.isin(multi_cabin)][["Cabin"]].sample(5)

- For the initials of a cabin in multi-cabin, some have the same initials and some are different. In different cases, the initials are extracted differently.

In [1]:
multi_cabin_title = []
for cabin in have_cabin[have_cabin.PassengerId.isin(multi_cabin)]["Cabin"].values:
    cabinA = cabin.split(" ")[0]
    cabinA_title = list(cabinA)[0]
    cabinB = cabin.split(" ")[1]
    cabinB_title = list(cabinB)[0]
    if cabinA_title==cabinB_title:
        multi_cabin_title = multi_cabin_title+[cabinA_title]
    else:
        multi_cabin_title = multi_cabin_title+[cabinA_title+cabinB_title]
        
have_cabin.loc[(have_cabin.PassengerId.isin(multi_cabin)),"Cabin_Title"] = multi_cabin_title
have_cabin[have_cabin.PassengerId.isin(multi_cabin)][["Cabin","Cabin_Title"]][:3]

In [1]:
#Add "Cabin_Title" to full dataset
cabin_title_list = list(have_cabin.Cabin_Title.values)
combine.loc[(combine.Cabin.notna()),"Cabin_Title"] = cabin_title_list
combine[combine.Cabin.notna()][["Cabin","Cabin_Title"]].head(2)

### 9."Fare_PP"
- Since a single ticket may contain more than one passenger, and the existing "Fare" indicates the price of a single ticket, the "Fare per person" can be extracted as another feature.

In [1]:
combine[["Fare","Ticket_Counts","Pclass"]].sample(5)

In [1]:
combine.loc[(combine.Fare.notna()),"Fare_PP"] = combine.Fare/combine.Ticket_Counts
combine[["Fare","Fare_PP","Ticket_Counts"]].sample(5)

## 2.2 Filling missing values<a id="5"></a>
- Since some features aren't important and rarely used,for missing values,we just make a simple filling.

#### 1."Fare"和"Fare_PP"

In [1]:
combine[combine.Fare.isna()]

- Since there only one missing value in "Fare", and "Fare" is associate with "Embarked" (distance) and "Pclass" (level), it is possible to use the median fare of the same "Embarked" and "Pclass" to filling the missing values

In [1]:
combine.loc[(combine.Fare_PP.isna()),"Fare_PP"] = combine[(combine.Embarked=="S")&(combine.Pclass==3)]["Fare_PP"].median()
combine.loc[(combine.Fare.isna()),"Fare"]  = combine[(combine.Embarked=="S")&(combine.Pclass==3)]["Fare_PP"].median()

#### 2."Cabin_Title"

In [1]:
#Ticket with multiple passengers and have cabin
multi_ticket = list(combine[(combine.Ticket_Counts>=2)&(combine.Cabin_Title.notna())]["Ticket"].values)
len(set(multi_ticket))

In [1]:
#With the same "Ticket" but different "Cabin_Title"
multi_ticket_cabin = combine[(combine.Cabin_Title.notna())&(combine.Ticket_Counts>=2)][["Ticket","Cabin_Title"]]
same_ticket_differ_cabin = []
for i,j in multi_ticket_cabin.groupby(["Ticket"]):
    differ_cabin = set(list(j.Cabin_Title.values))
    if len(differ_cabin)>=2:
        same_ticket_differ_cabin = same_ticket_differ_cabin+[i]
print(same_ticket_differ_cabin)

- Except in rare cases, passenger under the same ticket has the same "Cabin_Title", so we can use the "Cabin_Title" of the same ticket to filling the missing "Cabin_Title".

In [1]:
need_filling = list(combine[(combine.Ticket.isin(multi_ticket))&(combine.Cabin.isna())].PassengerId.values)
for passid in need_filling:
    ticket = list(combine[combine.PassengerId==passid]["Ticket"].values)
    cabin_title = combine[combine.Ticket.isin(ticket)]["Cabin_Title"].describe()["top"]
    combine.loc[combine.PassengerId==passid,"Cabin_Title"] = cabin_title
combine[combine.PassengerId.isin(need_filling)][["Cabin_Title"]][:3]

In [1]:
#remove repeated features
combine.drop(["Cabin","Fare"],axis = 1,inplace = True)

## 2.3 Modify error information<a id="6"></a>
- Since "Parch","SibSp","Family_size" are important features to group "ship team" and "second family",and we can find out through the internet that there are some people cancel the trip.This may result in an inconsistent number of registered relatives and the number of relatives on the ship.
- When the "Family_size" of a passenger is greater than or equal to 2，but there is nobody with the same surname or maiden name,or same ticket,when the relationship of the step-parent is also excluded, the passenger's information may be statistically incorrect.

### 2.3.1 Filter passengers who may have incorrect information

- There are no passengers of the same surname with a "Family_size" greater or equal to 2, and no passengers of the same maiden name with a "Family_size" greater or equal to, but the "Family_size" of  the passenger is greater or equal to 2. 

In [1]:
maybe_wrong = []
for i in range(combine.shape[0]):
    fz = combine.iloc[i]["Family_size"]
    sc = combine.iloc[i]["Surname_Counts"]
    surname = combine.iloc[i]["Surname"]
    maiden = combine.iloc[i]["Maiden"]
    passengerid = combine.iloc[i]["PassengerId"]
    maiden_count = combine[combine.PassengerId==(i+1)]["Maiden"].describe()["count"]
    if fz>sc:
        if maiden_count==0:
            sur_mai_family_count = combine[(combine.Maiden==surname)&(combine.Family_size>=2)].shape[0]
            if sur_mai_family_count==0:
                maybe_wrong = maybe_wrong+[passengerid]
        else:
            sur_mai_family_count = combine[(combine.Maiden==surname)&(combine.Family_size>=2)].shape[0]
            if sur_mai_family_count==0:
                mai_mai_family_count = combine[(combine.Maiden==maiden)&(combine.Family_size>=2)].shape[0]
                if mai_mai_family_count==0:
                    mai_sur_family_count = combine[(combine.Surname==maiden)&(combine.Family_size>=2)].shape[0]
                    if mai_sur_family_count==0:
                        maybe_wrong = maybe_wrong+[passengerid]
print(maybe_wrong)

### 2.3.2 Check these passenger's imformation

#### 1.Filtering step-parent relationship
-  If there is a step_parent relationship,they should have the same ticket,that is,"Ticket_Counts"should greater or equal to 2.

In [1]:
need_check = combine[(combine.PassengerId.isin(maybe_wrong))&(combine.Ticket_Counts==1)]["PassengerId"].values
need_check

#### 2.Check for maiden name
- check if there are extraction errors of passenger's maiden name 

In [1]:
def get_from_passid(dataset,passid_list):
    return dataset[dataset.PassengerId.isin(passid_list)][['Age', 'Cabin_Title', 'Embarked', 'Name', 'Parch', 'SibSp','Ticket',
                                                           'Ticket2', 'Surname','Maiden',"Ticket_Counts", 'Pclass_Gender',
                                                           'Family_size',"Survived"]]

def get_from_name(dataset,name_list):
    return dataset[(dataset.Surname.isin(name_list))
                   |(dataset.Maiden.isin(name_list))][['Age', 'Cabin_Title', 'Embarked', 'Name', 'Parch', 'SibSp','Ticket',
                                                           'Ticket2', 'Surname','Maiden',"Ticket_Counts", 'Pclass_Gender',
                                                           'Family_size',"Survived"]]

In [1]:
get_from_passid(combine,need_check)

In [1]:
get_from_name(combine,["Newsom","Monypeny"])

__[137] passenger have maiden  extraction errors__

#### 3.Based on other important features
- See if there are immediate family members based on the remaining important features，such as:"Cabin_Title","Embarked","Family_size","Pclass" and "Age".

In [1]:
#Check if  [274] have immediate family member
combine[(combine.Ticket2=="175xx")&(combine.Cabin_Title=="C")&(combine.Embarked=="C")&(combine.Family_size>=2)&(combine.Pclass==1)]

__According "Age","Parch" and "SibSp",Judging there is a statistical error of [274] passenger__

In [1]:
#Check if  [418&1130] have immediate family member
combine[((combine.Age<=3)|(combine.Age>=32))
        &(combine.Embarked=="S")&(combine.Family_size>=2)
        &(combine.Ticket2=="2506xx")&(combine.Pclass==2)][['Age', 'Cabin_Title', 'Embarked', 'Name', 'Parch', 'SibSp','Ticket',
                                                           'Ticket2', 'Surname','Maiden',"Ticket_Counts", 'Pclass_Gender', 'Family_size']]

__These two passenger also have statistical error__

In [1]:
#Check if  [443] have immediate family member
combine[(combine.Embarked=="S")&(combine.Ticket2=="3470xx")&(combine.Pclass==3)&(combine.Family_size==2)][['Age', 'Cabin_Title', 'Embarked', 'Name', 'Parch', 'SibSp','Ticket', 'Surname', 'Maiden', 
       'Pclass_Gender', 'Family_size', 'Surname_Counts', 'Ticket_Counts',"Maiden_Counts","Survived"]]

__[268&443] may be step brother,but not sure__

### 2.3.3 Modify incorrect information

In [1]:
combine.loc[combine.PassengerId.isin([137]),"Maiden"] = "Monypeny"
combine.loc[combine.Maiden.isin(["Monypeny"]),"Maiden_Counts"] = 2

combine.loc[combine.PassengerId.isin([274]),"Parch"] = 0
combine.loc[combine.PassengerId.isin([274]),"Family_size"] = 1

combine.loc[combine.PassengerId.isin([418]),"Parch"] = 0
combine.loc[combine.PassengerId.isin([418]),"Family_size"] = 1

combine.loc[combine.PassengerId.isin([1130]),"Parch"] = 0
combine.loc[combine.PassengerId.isin([1130]),"SibSp"] = 0
combine.loc[combine.PassengerId.isin([1130]),"Family_size"] = 1

# 3.Analysis of relationship between new features and results<a id="7"></a>

## 3.1 An overview of features and Survived<a id="8"></a>

In [1]:
%%HTML
<div class='tableauPlaceholder' id='viz1562742692050' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Titanic_15627427473480&#47;Overview2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Titanic_15627427473480&#47;Overview2' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ti&#47;Titanic_15627427473480&#47;Overview2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1562742692050');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

**Observation conclusion：**

**Aboout Overview 1**
- 1."Pclass_Gender" has a greater relationship with survival rate, "P1&P2 Boy" all survives, "P1&P2 Woman" has a high survival rate, and "P2 Man" has a low survival rate.
- 2.When "Fare_PP" is higher than 40, the survival rate is higher. However, due to the concentrated distribution of fares, "Fare_PP" can only be used as an auxiliary feature to predict survival.
- 3.The "Age" distribution is a normal distribution, except for the higher survival rate of the younger passengers, the survival rate of the other passengers is not much different, which is consistent with the conclusion 1.
- 4.Passengers embarked from "S port" have the highest mortality rate, and the survival rates of the other two ports are not much different,so it can only be used as an auxiliary feature to predict survival.
- 5."Cabin_Title" is also a normal distribution, and overall it has little to do with survival.

**About Overview 2**
- 1.The distribution of "Ticket_Counts" differs from the distribution of "Family_size", indicating that not all family members are in the same "ship team"。
- 2.The distribution of the four features is similar ,indicating that these features are related to each other.
- 3.Since their distribution shape are all with a very large positive skew, the relationship between these features and survival rate is not clear.

## 3.2 Check the survival rate of all types of passengers<a id="9"></a>

In [1]:
test_error = pd.DataFrame(columns=["Category","Train_data","Test_data","Error_Counts","Error_Probability"])

def get_error_counts(pg,index):
    PG_train_size = combine[(combine.Pclass_Gender.isin(pg))&(combine.Survived.notna())].shape[0]
    PG_test_size = combine[(combine.Pclass_Gender.isin(pg))&(combine.Survived.isna())].shape[0]
    PG_survival = combine[(combine.Pclass_Gender.isin(pg))]["Survived"].mean()
    if PG_survival>=0.5:
        error_probability = 1-PG_survival
        try:
            test_error_counts = round(error_probability*PG_test_size)
        except:
            test_error_counts=0
    else:
        error_probability = PG_survival
        test_error_counts = round(error_probability*PG_test_size)
    test_error.loc[index]=[pg,PG_train_size,PG_test_size,test_error_counts,"%.2f%%"%(round(PG_survival*100,2))]

index = 0
PG_list = [["P1-Woman","P2-Woman"],["P3-Woman"],["P1-Man"],["P2-Man"],["P3-Man"],["P1-Boy","P2-Boy"],["P3-Boy"]]
for pg in PG_list:
    get_error_counts(pg,index)
    index +=1
test_error.sort_values(by = "Error_Counts",ascending = False)

**Observation conclusion:**
- 1.The "Survived" of these passenger can be predicted only based on "Pclass_Gender": "P1&P2 Woman","P1&P2 Boy" and "P2-Man"
- 2.The survival rate of the remaining passengers needs to be predicted based on other features.

## 3.3 Survival prediction<a id="10"></a>

In [1]:
combine.loc[(combine.Pclass_Gender.isin(["P1-Boy","P2-Boy","P1-Woman","P2-Woman"])),"Predict"] = 1
combine.loc[(combine.Pclass_Gender=="P2-Man"),"Predict"] = 0

# 4."ship team" and "secondary family"<a id="11"></a>

**Noun definition：**
- 1."ship team"：A. Passengers with the same ticket. B. Immediate family members adjacent to the ticket. C. Cousins adjacent to the ticket (same surname or maiden, but not each other's immediate family).
- 2."secondary family"：Immediate family members of different "ship team" can form a secondary family

**Core method: layer by layer**

## 4.1 Passengers with immediate family members<a id="12"></a>

### 4.1.1 “Only with the immediate family members” or “including a secondary family”

**Step 1: Extract all passengers with immediate family members**

In [1]:
have_family = combine[combine.Family_size>=2]

##### Step 2: Check for abnormal passengers and add new features "Group_size" and "Ave_Survival" for them

In [1]:
have_family[have_family.Ticket2_Counts==1][['Age','Cabin_Title', 'Embarked','Pclass_Gender','Name',
                                        "Surname",'Maiden', 'Parch','SibSp', 'Family_size','Ticket',"Ticket_Counts"]]

In [1]:
get_from_name(have_family,["Olsen"])

- Conclusion: The two passengers are father-son relationship, but their ticket is quite different

**Note: When the "Ave_Survival" is 0 or 1, and the test data is included in the team, the real "Ave_Survival" needs to be checked again. The rules are as follows:**
- When "Ave_Survival" is 0, in both cases, the real "Ave_Survival" is 0: A. Train data include women. B.Test data in team are all men.
- When "Ave_Survival" is 1, in both cases, the real "Ave_Survival" is 1: A. Train data include men. B.Test data in team are all women. 
- In addition to these cases, the "Ave_Survival" was set at 0.5.

In [1]:
def get_team_id(dataset,name_list):
    return dataset[(dataset.Surname.isin(name_list))|(dataset.Maiden.isin(name_list))]["PassengerId"].values

In [1]:
team_id =get_team_id(have_family,["Olsen"])
have_family.loc[have_family.PassengerId.isin(team_id),"Group_size"] = have_family[have_family.PassengerId.isin(team_id)].shape[0]
have_family.loc[have_family.PassengerId.isin(team_id),"Ave_Survival"] = have_family[have_family.PassengerId.isin(team_id)]["Survived"].mean()

In [1]:
get_from_name(have_family,["Crosby","Halstead"])

- Conclusion: These three passengers are the relationship between parents and daughters

In [1]:
team_id = get_team_id(have_family,["Crosby","Halstead"])
have_family.loc[have_family.PassengerId.isin(team_id),"Group_size"] = have_family[have_family.PassengerId.isin(team_id)].shape[0]
have_family.loc[have_family.PassengerId.isin(team_id),"Ave_Survival"] = have_family[have_family.PassengerId.isin(team_id)]["Survived"].mean()

**Step 3: Divide the remaining passengers into two categories: “Only with the immediate family members” or “including a secondary family”**

<img src="https://raw.githubusercontent.com/liuqiantx/Mechine-learning/master/Titanic/Visio.png">

**Note**
- team1:when "Family_size" is greater than "Ticket_Counts",all passengers with the same name (family surname or maiden name) from the passenger in the fuzzy ticket number(adjacent or same ticket number)
- team2:when "Family_size" is less than or equal to "Ticket_Counts",all passengers with the same name (family surname or maiden name) from the passenger in the same ticket number.
- team3:when "Family_size" is less than or equal to "Ticket_Counts",all passengers with the same name (family surname or maiden name) from the passenger in the fuzzy ticket number(adjacent or same ticket number).

In [1]:
have_secondary_family1 = []
have_secondary_family2 = []
ship_team_1 = []
ship_team_2 = []
ship_team_2_tic2 = []
for passengerid in have_family.PassengerId.values:
    
    #Extracting some feature values of the passenger to identify same"ship team" and same"family" 
    ticket = have_family[have_family.PassengerId==passengerid]["Ticket"].describe()["top"]
    ticket_counts = have_family[have_family.PassengerId==passengerid]["Ticket_Counts"].describe()["mean"]
    family_size = have_family[have_family.PassengerId==passengerid]["Family_size"].describe()["mean"]
    ticket2 = have_family[have_family.PassengerId==passengerid]["Ticket2"].describe()["top"]
    surname = have_family[have_family.PassengerId==passengerid]["Surname"].describe()["top"]
    maiden = set(have_family[(have_family.PassengerId==passengerid)&(have_family.Maiden.notna())]["Maiden"].values)
    
    #When "Family_size" is greater than Ticket_Counts",
    #it may be that some family members exist in the adjacent ticket number.(it still the same "ship team")
    if family_size>ticket_counts:
        
        #Extract all passengers with the same name (family surname or maiden name) from the passenger 
        #in the fuzzy ticket number (adjacent or same ticket number)
        group_by_tic2 = have_family[((have_family.Surname.isin({surname}|maiden))
                                                  |(have_family.Maiden.isin({surname}|maiden)))
                                                 &(have_family.Ticket2.isin({ticket2}))]
        group_by_tic2_size = group_by_tic2.shape[0]
        if group_by_tic2_size == family_size:
            ship_team_1 = ship_team_1+[passengerid]
        else:
            #If the team is smaller than the "Family_size", the passenger has a secondary family
            have_secondary_family1 = have_secondary_family1+[passengerid]
       
    #When the ""Family_size" is equal to the "Ticket_Counts", 
    #first check if all passengers with the same ticket number are family members,
    #if not, continue to check if all family members are in the adjacent ticket number.
    
    #When the ""Family_size" is less than the "Ticket_Counts",
    #First check if the number of family members with the same ticket number is equal to the total number of family members
    #(with family and friends traveling together)
    #if not, continue to check if all family members are in the adjacent ticket number.
    else:
        group_by_tic = have_family[((have_family.Surname.isin({surname}|maiden))|(have_family.Maiden.isin({surname}|maiden)))
                                      &(have_family.Ticket.isin({ticket}))]
        group_by_tic2 = have_family[((have_family.Surname.isin({surname}|maiden))
                                     |(have_family.Maiden.isin({surname}|maiden)))
                                    &(have_family.Ticket2.isin({ticket2}))]
        group_by_tic_size = group_by_tic.shape[0]
        group_by_tic2_size = group_by_tic2.shape[0]
        if group_by_tic_size == family_size:
            ship_team_2 = ship_team_2+[passengerid]
        elif group_by_tic2_size == family_size:
            ship_team_2_tic2 = ship_team_2_tic2+[passengerid]
        else:
            have_secondary_family2 = have_secondary_family2+[passengerid]
print(have_secondary_family1)
print("-"*50)
print(have_secondary_family2)

**Step 4: Filling the "Group_size" and "Ave_Survival" of the passengers who "only with their immediate relatives"**

**Two situations**
- Situation 1: All immediate family members are in the same ticket number (ship_family_2). In this situation, the "Group_size" is the size of the "team consisting of all passengers with the same ticket number and the same name", and the "Ave_Survival" is the average survival of all passengers in the same ticket in the complete train data.
- Situation 2:All immediate family members are in the fuzzy ticket(ship_family_2_tic2, ship_family_1).In this situation, the "Group_size" is the average "Family_size" of all passengers in the same fuzzy ticket and same name.and the "Ave_Survival" is the average survival of all passengers in the team ticket in the complete train data.

In [1]:
def full_gs_and_as(dataset,id_list,ticket_kind,check_list,with_friend,count_maiden):
    for i in id_list:
        surname = set(dataset[dataset.PassengerId.isin([i])]["Surname"].values)
        ticket_list = set(dataset[dataset.PassengerId.isin([i])][ticket_kind].values)
        
        #For passengers with immediate family members, 
        #the identification of “ship team” and “secondary family” requires the use of “maiden name”
        if count_maiden ==True:
            maiden = set(dataset[dataset.PassengerId.isin([i])&(dataset.Maiden.notna())].Maiden.values)
            team = dataset[((dataset.Surname.isin(surname|maiden))
                        |(dataset.Maiden.isin(surname|maiden)))
                       &(dataset[ticket_kind].isin(ticket_list))]
            team_pg = team.Pclass_Gender.describe().top
            
            #When the "ship team" is identified by the same ticket number
            if ticket_kind=="Ticket":
                team_size = team.shape[0]
                ave_survival = dataset[dataset[ticket_kind].isin(ticket_list)].Survived.mean()
                team_passid = list(team.PassengerId.values)
                
            #When the "ship team" is identified by the fuzzy ticket number(Same or adjacent ticket number)
            else:
                team_size = round(team.Family_size.describe().mean())
                ticket = set(team.Ticket.values)
                ave_survival = dataset[dataset.Ticket.isin(ticket)].Survived.mean()
                team_passid = list(team.PassengerId.values)
                
        #To identify the members of the same "ship team"  which only consists of cousins, it is not necessary to use “maiden name” 
        #(I will talk about it later, not used here)
        else:
            team = dataset[(dataset[ticket_kind].isin(ticket_list))&(dataset.Surname.isin(surname))&(dataset.Group_size.isna())]
            team_size = team.shape[0]
            if team_size >=1:
                team_pg = team.Pclass_Gender.describe().top
            else:
                team_pg = []
            if team_size>=2:
                ticket = set(team.Ticket.values)
                ticket_team_size = dataset[dataset.Ticket.isin(ticket)].shape[0]
                if team_size == ticket_team_size:
                    team_passid = list(team.PassengerId.values)
                    ave_survival=team.Survived.mean()
                else:
                    team_passid = []
                    ave_survival = nan
                    with_friend = with_friend+[i]
            else:
                team_passid = []
                ave_survival = nan
                
        #Number of test data and train data in the team
        empty_survived = dataset[(dataset.PassengerId.isin(team_passid))&(dataset.Survived.isna())].shape[0]
        full_survived = dataset[(dataset.PassengerId.isin(team_passid))&(dataset.Survived.notna())].shape[0]
        
        #The following loop is consistent with the judgment rule about True "Ave_Survival" in step 2
        if full_survived>0:
            if empty_survived>0:
                empty_gender_count = team[(team.Survived.isna())].Gender.describe()["unique"]
                empty_gender = set(team[(team.Survived.isna())].Gender.values)
                full_gender = set(team[(team.Survived.notna())].Gender.values)
                if ave_survival ==0:
                    if ("Woman" in full_gender)|(("Man" in empty_gender)&(empty_gender_count==1)):
                        dataset.loc[dataset.PassengerId.isin(team_passid),"Ave_Survival"] = 0
                    else:
                        dataset.loc[dataset.PassengerId.isin(team_passid),"Ave_Survival"] = 0.5
                elif ave_survival==1:
                    if ("Man" in full_gender)|(("Woman" in empty_gender)&(empty_gender_count==1)):
                        dataset.loc[dataset.PassengerId.isin(team_passid),"Ave_Survival"] = 1
                    else:
                        dataset.loc[dataset.PassengerId.isin(team_passid),"Ave_Survival"] = 0.5
                
                #When the training data is all women, even if the average survival rate of the team is greater than 0.5, 
                #it cannot be judged that the survival rate of the men in the team is greater than 0.5
                #When the training data is all men, even if the average survival rate of the team is less than 0.5, 
                #it cannot be judged that the survival rate of the women in the team is less than 0.5
                else:
                    check_list=check_list+[team_passid]
            else:
                dataset.loc[(dataset.PassengerId.isin(team_passid)),"Ave_Survival"]=ave_survival
        else:
            dataset.loc[(dataset.PassengerId.isin(team_passid)),"Ave_Survival"]=nan
        dataset.loc[(dataset.PassengerId.isin(team_passid)),"Group_size"] = team_size
    print("These passenger travel not only with cousin:",with_friend)
    check_list = list(np.unique(check_list))
    print("these passenger need check more:",check_list)

In [1]:
full_gs_and_as(dataset = have_family,id_list=ship_team_2,ticket_kind = "Ticket",check_list = [],with_friend=[],count_maiden = True)

In [1]:
#Since the survival rate of the P2 passengers has been predicted by "Pclass_Gender", 
#the misjudgment of the average survival rate of such passengers may not be considered.
def fillfull_p2(dataset,check_list,check_more):
    for i in check_list:
        pclass = dataset[dataset.PassengerId.isin(i)].Pclass.describe()["mean"]
        if pclass != 2:
            check_more = check_more+[i]
        else:
            dataset.loc[dataset.PassengerId.isin(i),"Group_size"] = len(i)
            dataset.loc[dataset.PassengerId.isin(i),"Ave_Survival"] = dataset[dataset.PassengerId.isin(i)]["Survived"].mean()
    return print(check_more)

In [1]:
check_list = [[26, 183, 234, 262, 1046, 1066, 1271], [28, 89, 342, 439, 945, 961], [44, 609, 686, 1188], [59, 451, 473, 1142],
              [94, 789, 924, 1246], [119, 300, 1076], [246, 413, 1303], [280, 747, 1284], [298, 306, 499, 1198], [671, 685, 1067], 
              [818, 828, 1253]]
fillfull_p2(dataset = have_family,check_list = check_list,check_more = [])

In [1]:
def check_ave_survival(dataset,passid):
    return dataset[dataset.PassengerId.isin(passid)][["Pclass_Gender","Survived"]]

def fullfill_ave_survival(dataset,check_list,check_more):
    for i in check_list:
        ave_survival = dataset[dataset.PassengerId.isin(i)].Survived.mean()
        empty_gender = list(dataset[(dataset.PassengerId.isin(i))&(dataset.Survived.isna())].Gender.values)
        if ave_survival>0.5:
            if "Man" in empty_gender:
                dataset.loc[(dataset.PassengerId.isin(i)),"Ave_Survival"] = 0.5
            else:
                dataset.loc[(dataset.PassengerId.isin(i)),"Ave_Survival"] = ave_survival
        elif ave_survival==0.5:
            if "Woman"in empty_gender:
                dataset.loc[(dataset.PassengerId.isin(i)),"Ave_Survival"] = 0.5
            else:
                dataset.loc[(dataset.PassengerId.isin(i)),"Ave_Survival"] = ave_survival
        else:
            check_more = check_more+[i]
    return print(check_more)

In [1]:
check_list = [[26, 183, 234, 262, 1046, 1066, 1271], [28, 89, 342, 439, 945, 961], [94, 789, 924, 1246],
              [119, 300, 1076], [246, 413, 1303], [280, 747, 1284], [298, 306, 499, 1198]]
fullfill_ave_survival(have_family,check_list,check_more = [])

In [1]:
check_ave_survival(have_family,[298, 306, 499, 1198])

In [1]:
have_family.loc[(have_family.PassengerId.isin([298, 306, 499, 1198])),"Ave_Survival"] = 0.33

In [1]:
situation_2 = ship_team_1+ship_team_2_tic2
full_gs_and_as(dataset = have_family,id_list=situation_2,ticket_kind = "Ticket2",check_list = [],with_friend=[],count_maiden = True)

In [1]:
check_list = [[70, 185, 1057, 1268, 1286], [408, 438, 530, 775, 832, 944], [438, 530, 775, 944], [477, 727, 923, 1211], 
              [672, 821, 984, 1200], [727, 923, 1211], [821, 984, 1200]]
fillfull_p2(have_family,check_list,check_more = [])

In [1]:
check_list = [[70, 185, 1057, 1268, 1286], [672, 821, 984, 1200], [821, 984, 1200]]
fullfill_ave_survival(have_family,check_list,check_more=[])

**Step 5: Filling the "Group_size" and "Ave_Survival" of the passengers who have "secondary families" **

In [1]:
remain_second_family = list(have_family[(have_family.PassengerId.isin(have_secondary_family1+have_secondary_family2))
                                        &(have_family.Group_size.isna())]["PassengerId"].values)
print(len(remain_second_family))

In [1]:
#Some passengers have a secondary family, but only the first-class family members are used as the basis for "Group_size" and "Ave_Survival".
#That is, although the passengers of the same ticket are all immediate family members, some relatives are not in the same ticket.
second_family = have_secondary_family1+have_secondary_family2
for i in remain_second_family:
    second_family.remove(i)
print(second_family)

In [1]:
check_list = []
for i in second_family:
    pclass = have_family[have_family.PassengerId==i].Pclass.describe().max
    if pclass != 2:
        surname = list(have_family[have_family.PassengerId.isin([i])].Surname.values)
        try:
            maiden = list(have_family[have_family.PassengerId.isin([i])].Maiden.values)
        except:
            maiden = []
        ticket2 = list(have_family[have_family.PassengerId.isin([i])].Ticket2.values)
        old_team_size = have_family[have_family.PassengerId.isin([i])].Group_size.describe()["mean"]
        new_team = have_family[(have_family.Ticket2.isin(ticket2))
                               &((have_family.Surname.isin(surname+maiden))|(have_family.Maiden.isin(surname+maiden)))]
        new_team_size = new_team.shape[0]
        new_team_passid = list(new_team.PassengerId.values)
        ave_survival = new_team.Survived.mean()
        
        #The following loop is consistent with the previous one
        full_survived = new_team[new_team.Survived.notna()].shape[0]
        empty_survived = new_team[new_team.Survived.isna()].shape[0]
        if new_team_size>old_team_size:
            if full_survived>0:
                if empty_survived>0:
                    empty_gender_count = new_team[(new_team.Survived.isna())].Gender.describe()["unique"]
                    empty_gender = set(new_team[(new_team.Survived.isna())].Gender.values)
                    full_gender = set(new_team[(new_team.Survived.notna())].Gender.values)
                    if ave_survival ==0:
                        if ("Woman" in full_gender)|(("Man" in empty_gender)&(empty_gender_count==1)):
                            have_family.loc[have_family.PassengerId.isin(new_team_passid),"Ave_Survival"] = 0
                        else:
                            have_family.loc[have_family.PassengerId.isin(new_team_passid),"Ave_Survival"] = 0.5
                    elif ave_survival==1:
                        if ("Man" in full_gender)|(("Woman" in empty_gender)&(empty_gender_count==1)):
                            have_family.loc[have_family.PassengerId.isin(new_team_passid),"Ave_Survival"] = 1
                        else:
                            have_family.loc[have_family.PassengerId.isin(new_team_passid),"Ave_Survival"] = 0.5
                    else:
                        check_list=check_list+[new_team_passid]
                else:
                    have_family.loc[(have_family.PassengerId.isin(new_team_passid)),"Ave_Survival"]=ave_survival
            else:
                have_family.loc[(have_family.PassengerId.isin(new_team_passid)),"Ave_Survival"]=nan
        have_family.loc[have_family.PassengerId.isin(new_team_passid),"Group_size"] = new_team_size
print(check_list)

In [1]:
have_family[have_family.PassengerId.isin([70, 185, 1057, 1268, 1286])][["Pclass_Gender","Survived","Ave_Survival"]]

**The “secondary family” of the remaining 30 passengers is divided into three situations:**
- Possibility #1: There is a step-parent relationship. That is, the family size of all members with the same ticket is equal to the size of this team.
- Possibility #2: There are immediate family members on ship, but their ticket is very different.That is,The size of the team which consists of all members with same name among the remaining passengers is equal to the "Family_size" of the team members.
- Possibility #3：If the above conditions are not met, it is necessary to use more feature values of the passengers, or it may be a statistical error of the passenger information.

###### 1.Stepparent relationship

In [1]:
next_check = []
have_stepparent=[]
for i in remain_second_family:
    ticket = have_family[have_family.PassengerId==i].Ticket.describe()["top"]
    team = have_family[have_family.Ticket.isin([ticket])]
    team_passid = list(team.PassengerId.values)
    team_size = team.shape[0]
    family_size = team.Family_size.describe()["max"]
    if team_size == family_size:
        have_family.loc[(have_family.PassengerId.isin(team_passid)),"Ave_Survival"] = team.Survived.mean()
        have_family.loc[(have_family.PassengerId.isin(team_passid)),"Group_size"] = team_size
        have_stepparent=have_stepparent+[team_passid]
    else:
        next_check = next_check+[i]
have_stepparent = np.unique(have_stepparent)
print("these passengers are stepparent relationship:",have_stepparent)
print("these passengers need to be checked more:",next_check)

###### 2.Immediate relatives with large difference in ticket number

In [1]:
next_check_2 = []
have_grouped = []
for i,j in have_family[have_family.PassengerId.isin(next_check)].groupby(["Surname"]):
    if (len(j))>=2:
        family_size = j.Family_size.mean()
        j_size = j.shape[0]
        j_passid = list(j.PassengerId.values)
        if j_size == family_size:
            have_grouped = have_grouped+[j_passid]
            have_family.loc[(have_family.PassengerId.isin(j_passid)),"Group_size"] = j_size
            have_family.loc[(have_family.PassengerId.isin(j_passid)),"Ave_Survival"] = j.Survived.mean()
        else:
            next_check_2 = next_check_2+[j_passid]
    else:
        j_passid = list(j.PassengerId.values)
        next_check_2 = next_check_2+[j_passid]
print("have grouped",have_grouped)
print("next more check",next_check_2)

###### 3.Use "Embarked", "Ticket2", "Pclass" to group the remaining passengers

In [1]:
need_more_check = [41,193,69,1106,581,1133,1296,248,756,722,313,1041,443,893]
can_groupby_ticket2 = []
maybe_wrong = []
for i,j in have_family[have_family.PassengerId.isin(need_more_check)].groupby(["Embarked","Ticket2","Pclass"]):
    if len(j)>=2:
        can_groupby_ticket2 = can_groupby_ticket2+[i[1]]
    else:
        maybe_wrong = maybe_wrong+[i[1]]
print("can groupby ticket2",can_groupby_ticket2)
print("maybe wrong",maybe_wrong)

###### 3.1 Check the team group by fuzzy ticket.

In [1]:
def get_from_id_tic2(dataset,passid,tic2):
    return dataset[(dataset.PassengerId.isin(passid))
                   &(dataset.Ticket2.isin(tic2))][['Age', 'Embarked', 'Parch', 'SibSp','Ticket','Surname', 'Maiden','Family_size',
                                                   'Pclass_Gender','Cabin_Title',"Survived","Ave_Survival"]]

In [1]:
get_from_id_tic2(have_family,need_more_check,["2377xx"])

In [1]:
have_family.loc[(have_family.PassengerId.isin([581,1133])),["Group_size","Ave_Survival"]] = [2,1]

- [581, 1133] is a "ship team", but there is a child of 1133, a brother of sister of 581 canceled boarding.

In [1]:
get_from_id_tic2(have_family,need_more_check,['2506xx'])

- [248,313,756,1041] is a "ship team", but there is a brother of sister of 756 and a child of 581 canceled boarding.

In [1]:
have_family.loc[(have_family.PassengerId.isin([248,313,756,1041])),["Group_size","Ave_Survival"]] = [3,0.5]

In [1]:
get_from_id_tic2(have_family,need_more_check,['3470xx'])

- There is no evidence to prove that there is a brother and sister relationship between [443,1106].especially [1106] has a large family.

In [1]:
get_from_id_tic2(have_family,need_more_check,['3500xx'])

In [1]:
have_family.loc[(have_family.PassengerId.isin([193,722])),["Group_size","Ave_Survival"]] = [2,0.5]

- there is a brother and sister relationship between [193,722]

In [1]:
have_family.loc[have_family.PassengerId==443,["Group_size",'Ave_Survival']] = [2,0]
have_family.loc[have_family.PassengerId==1106,["Group_size",'Ave_Survival']] = [7,0]

###### 3.2 Check the information of the remaining passengers

In [1]:
get_from_id_tic2(have_family,need_more_check,maybe_wrong)

- After considering multiple features comprehensively, passenger [41,893,1296] have a statistical error
- Passenger [69] is in a large family.

In [1]:
have_family.loc[have_family.PassengerId.isin([41,893,1296]),["SibSp","Family_size"]] = [0,1]
combine.loc[combine.PassengerId.isin([41,893,1296]),["SibSp","Family_size"]] = [0,1]
have_family.loc[have_family.PassengerId.isin([69,1106]),["Group_size",'Ave_Survival']] = [7,0.125]

**Step 6: Add "Group_size" and "Ave_Survival" of all passengers with immediate family members to the complete dataset**

In [1]:
have_family = have_family[have_family.Family_size!=1]
passid_list = list(have_family.PassengerId.values)
group_size_list = list(have_family.Group_size.values)
ave_survival = list(have_family.Ave_Survival.values)
combine.loc[combine.PassengerId.isin(passid_list),"Group_size"] = group_size_list
combine.loc[combine.PassengerId.isin(passid_list),"Ave_Survival"] = ave_survival

**Summary**
the "Group_size" and "Ave_Survival" of  all “passengers with immediate family members” have been fully filled. However, among the single passengers, there still could have a “ship team” that consists of cousins, friends, or couples etc.
- Traveling with cousins: same name, adjacent ticket number, and non-immediate relatives
- Friends and other non-relative relationships: same ticket number,and non-immediate relatives

## 4.2 Identify the "ship team"<a id="13"></a>

### 4.2.1 Traveling with cousins
- The identification method of the cousin:same surname,same maiden name,or one's surname is the same as another person's maiden name. 

#### Step 1：Filter passengers who traveling with cousins

- 1.same surname

In [1]:
all_passenger = []
for i,j in combine[(combine.Surname_Counts>=2)].groupby(["Ticket2","Surname"]):
    if len(j)>=2:
        j_passid = list(j.PassengerId.values)
        all_passenger = all_passenger+j_passid
same_surname = list(combine[(combine.PassengerId.isin(all_passenger))&(combine.Group_size.isna())]["PassengerId"].values)
print(len(same_surname))

- 2.same maiden name

In [1]:
#maiden
all_passenger = []
for i,j in combine[(combine.Maiden_Counts>=2)].groupby(["Ticket2","Maiden"]):
    if len(j)>=2:
        j_passid = list(j.PassengerId.values)
        all_passenger = all_passenger+j_passid
same_maiden = list(combine[(combine.PassengerId.isin(all_passenger))&(combine.Group_size.isna())]["PassengerId"].values)
print(same_maiden)

#### Step 2: Filter passengers travelling with cousins who have “Immediate family”

In [1]:
same_surname_ticket = list(combine[(combine.PassengerId.isin(same_surname))].Ticket.values)
combine[(combine.Ticket.isin(same_surname_ticket))&(combine.Group_size.notna())][['Age', 'Embarked', 'Parch', 'SibSp','Ticket',
                                                                                             'Surname', 'Maiden','Family_size',
                                                                                             'Pclass_Gender','Cabin_Title',"Surname_Counts","Group_size"]]

In [1]:
combine[combine.Ticket=="31352"][['Age', 'Embarked', 'Parch', 'SibSp','Ticket','Surname', 'Maiden','Family_size','Pclass_Gender',
                                  'Cabin_Title',"Surname_Counts","Group_size","Survived"]]

In [1]:
team = combine[(combine.Ticket=="31352")|((combine.Surname=="Ware")&(combine.Family_size!=1))]
combine.loc[(combine.PassengerId.isin([1170,1254])),"Group_size"] = 2
combine.loc[(combine.PassengerId.isin([1170,1254])),"Ave_Survival"] = team.Survived.mean()

#### Step 3: Divide these passengers into two categories: “only with the cousins” and “with non-cousin passengers in the ship team”,and filling the "Group_size" and the "Ave_Survival" of them.

In [1]:
same_surname.remove(1254)
full_gs_and_as(dataset = combine,id_list=same_surname,ticket_kind = "Ticket2",check_list = [],with_friend=[],count_maiden = False)

In [1]:
combine[(combine.PassengerId.isin([115, 245, 303, 598, 693, 827]))][["Pclass_Gender","Survived","Ticket"]]

In [1]:
combine.loc[(combine.Ticket.isin(["2627","2694"])),"Group_size"] = combine[(combine.Ticket.isin(["2627","2694"]))].shape[0]
combine.loc[(combine.Ticket.isin(["2627","2694"])),"Ave_Survival"] = combine[(combine.Ticket.isin(["2627","2694"]))]["Survived"].mean()
combine.loc[(combine.Ticket.isin(["LINE"])),"Group_size"] = combine[(combine.Ticket.isin(["LINE"]))].shape[0]
combine.loc[(combine.Ticket.isin(["LINE"])),"Ave_Survival"] = combine[(combine.Ticket.isin(["LINE"]))]["Survived"].mean()
combine.loc[(combine.Ticket.isin(["1601"])),"Group_size"] = combine[(combine.Ticket.isin(["1601"]))].shape[0]
combine.loc[(combine.Ticket.isin(["1601"])),"Ave_Survival"] = combine[(combine.Ticket.isin(["1601"]))]["Survived"].mean()

#### Step 4：one person's surname is the same as another person's maiden name

In [1]:
surname_list = list(combine.Surname.values)
common_name = []
for maiden in combine.Maiden.values:
    if maiden in surname_list:
        common_name = common_name+[maiden]
print(len(common_name))
print(common_name[:5])

In [1]:
possible_cousin = combine[((combine.Surname.isin(common_name))|(combine.Maiden.isin(common_name)))
                          &(combine.Family_size==1)&(combine.Group_size.isna())]
possible_cousin_id = []
for i,j in possible_cousin.groupby(["Ticket2"]):
    if len(j)>=2:
        j_passid = list(j.PassengerId.values)
        possible_cousin_id = possible_cousin_id+[j_passid]
print(possible_cousin_id)

- For the above groups，if one's surname is the same with another persons' maiden name in the same group,than they can group a "ship team".

In [1]:
combine[(combine.PassengerId.isin([41, 104, 714, 877]))][["Surname","Maiden"]]

- Only [41] maiden is the same with [714] surname,so they can group a "ship team"

In [1]:
combine[(combine.PassengerId.isin([41, 714]))][['Age',"Cabin_Title", 'Embarked','Pclass_Gender',"Surname",'Maiden','Ticket',"Survived"]]

In [1]:
combine.loc[combine.PassengerId.isin([41, 714]),["Group_size",'Ave_Survival']] = [2,0]

#### Step 5：Filling  the "Group_size" and "Ave_survival" of  the remaining "ship team"(group by the same ticket)

In [1]:
remain_passenger = combine[(combine.Group_size.isna())&(combine.Ticket_Counts>=2)]
remain_ticket = list(remain_passenger.Ticket.values)
check_list = []
for i in remain_ticket:
    team = combine[(combine.Ticket.isin([i]))]
    empty_size = combine[(combine.Ticket.isin([i]))&(combine.Survived.isna())].shape[0]
    full_size = combine[(combine.Ticket.isin([i]))&(combine.Survived.notna())].shape[0]
    team_passid = list(team.PassengerId.values)
    team_size = team.shape[0]
    ave_survival = team.Survived.mean()
    if full_size>0:
        if empty_size>0:
            empty_gender_count = combine[(combine.PassengerId.isin(team_passid))&(combine.Survived.isna())].Gender.describe()["unique"]
            empty_gender = set(combine[(combine.PassengerId.isin(team_passid))&(combine.Survived.isna())].Gender.values)
            full_gender = set(combine[(combine.PassengerId.isin(team_passid))&(combine.Survived.notna())].Gender.values)
            if ave_survival ==0:
                if ("Woman" in full_gender)|(("Man" in empty_gender)&(empty_gender_count==1)):
                    combine.loc[combine.PassengerId.isin(team_passid),"Ave_Survival"] = 0
                else:
                    combine.loc[combine.PassengerId.isin(team_passid),"Ave_Survival"] = 0.5
            elif ave_survival==1:
                if ("Man" in full_gender)|(("Woman" in empty_gender)&(empty_gender_count==1)):
                    combine.loc[combine.PassengerId.isin(team_passid),"Ave_Survival"] = 1
                else:
                    combine.loc[combine.PassengerId.isin(team_passid),"Ave_Survival"] = 0.5
            else:
                check_list=check_list+[team_passid]
        else:
            combine.loc[(combine.PassengerId.isin(team_passid)),"Ave_Survival"]=ave_survival
    else:
        combine.loc[(combine.PassengerId.isin(team_passid)),"Ave_Survival"]=nan
    combine.loc[(combine.PassengerId.isin(team_passid)),"Group_size"] = team_size
check_list = list(np.unique(check_list))
print(check_list)

In [1]:
check_list = [[146, 550, 1068, 1222], [270, 326, 374, 1206], [280, 747, 1065, 1284], [291, 742, 988], [298, 306, 499, 709, 1033, 1198],
              [308, 506, 1306], [381, 558, 701, 717, 1094], [538, 545, 1131]]
fillfull_p2(combine,check_list,check_more = [])

In [1]:
check_list = [[270, 326, 374, 1206], [280, 747, 1065, 1284], [291, 742, 988], [298, 306, 499, 709, 1033, 1198], [308, 506, 1306],
              [381, 558, 701, 717, 1094], [538, 545, 1131]]
fullfill_ave_survival(combine,check_list,check_more=[])

**Summary：so far，the "Group_size" and "Ave_survival" of all non-single passengers are filled.**

## 4.3 Add new feature “If_Survived”<a id="14"></a>

### 4.3.1 When "Ave_Survival" is equal to 0.5

In [1]:
#P3-Man
survival = combine[(combine.Pclass_Gender=="P3-Man")&(combine.Ave_Survival==0.5)]["Survived"].mean()
shape = combine[(combine.Pclass_Gender=="P3-Man")&(combine.Ave_Survival==0.5)&(combine.Survived.notna())].shape[0]
print(survival)
print(shape)

In [1]:
#P1-Man
survival = combine[(combine.Pclass_Gender=="P1-Man")&(combine.Ave_Survival==0.5)]["Survived"].mean()
shape = combine[(combine.Pclass_Gender=="P1-Man")&(combine.Ave_Survival==0.5)&(combine.Survived.notna())].shape[0]
print(survival)
print(shape)

In [1]:
#P3-Woman
survival =combine[(combine.Pclass_Gender=="P3-Woman")&(combine.Ave_Survival==0.5)]["Survived"].mean()
shape = combine[(combine.Pclass_Gender=="P3-Woman")&(combine.Ave_Survival==0.5)&(combine.Survived.notna())].shape[0]
print(survival)
print(shape)

In [1]:
#P3-Boy
survival =combine[(combine.Pclass_Gender=="P3-Boy")&(combine.Ave_Survival==0.5)]["Survived"].mean()
shape = combine[(combine.Pclass_Gender=="P3-Boy")&(combine.Ave_Survival==0.5)&(combine.Survived.notna())].shape[0]
print(survival)
print(shape)

- 当"Ave_Survival"为0.5时，"P1-Man" & "P3-Man" can be predicted as died，"P3-Woman"can be predicted as survived，but for "P3-Boy",it needs more check.

In [1]:
combine[(combine.Pclass_Gender=="P3-Boy")&(combine.Ave_Survival==0.5)
        &(combine.Survived.isna())][["PassengerId","Ticket","Ticket_Counts"]]

In [1]:
combine[combine.Ticket=="2673"][["PassengerId","Age","Pclass_Gender","Survived","Ave_Survival"]]

- According to human nature, when a mother took a son and no one in the group died, the son would survived,so [1053,1199,1309] will be predicted as survived.
- For the remaining boys,since someone in the team has died，so they will be predicted as died.

In [1]:
combine.loc[(combine.Pclass_Gender.isin(["P3-Woman"]))&(combine.Ave_Survival>=0.5),"If_Survived"] = 1
combine.loc[(combine.Pclass_Gender.isin(["P3-Woman"]))&(combine.Ave_Survival<0.5),"If_Survived"] = 0

combine.loc[(combine.Pclass_Gender.isin(["P1-Man","P3-Man"])&(combine.Ave_Survival>0.5)),"If_Survived"] = 1
combine.loc[(combine.Pclass_Gender.isin(["P1-Man","P3-Man"])&(combine.Ave_Survival<=0.5)),"If_Survived"] = 0

combine.loc[((combine.Pclass_Gender.isin(["P3-Boy"]))&(combine.Ave_Survival>0.5)),"If_Survived"] = 1
combine.loc[((combine.Pclass_Gender.isin(["P3-Boy"]))&(combine.Ave_Survival<0.5)),"If_Survived"] = 0

combine.loc[(combine.PassengerId.isin([1053,1199,1309])),"If_Survived"] = 1
combine.loc[(combine.PassengerId.isin([1046,1084,1231,1236,1271,1284])),"If_Survived"] = 0

combine.loc[(combine.Group_size.isna()),"Group_size"] = 1

## 4.4 An overview of "If_Survived" and "Survived"<a id="15"></a>

In [1]:
%%HTML
<div class='tableauPlaceholder' id='viz1563324785218' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ov&#47;OverviewoftheRelationshipBetweenIf_SurvivedandSurvived&#47;OverviewoftheRelationshipBetweenIf_SurvivedandSurvived&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='OverviewoftheRelationshipBetweenIf_SurvivedandSurvived&#47;OverviewoftheRelationshipBetweenIf_SurvivedandSurvived' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ov&#47;OverviewoftheRelationshipBetweenIf_SurvivedandSurvived&#47;OverviewoftheRelationshipBetweenIf_SurvivedandSurvived&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1563324785218');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

**Observation conclusion：**
- 1.For "P1-Man" and "P3-Man", due to their higher mortality in the complete dataset，especially for "P3-Man"，when making predictions based on "IF_Survived", although some passengers can be predicted to survive, their error rate is greater than 20%，and the sample size is small，it may have errors, so，for "P1-Man" and "P3-Man" in test data and be predicted to survive,it needs more check.
- 2.For "P3-Boy",since the prediction accuracy rate for them is 100%. the prediction results are credible.
- 3.For "P3-Woman":since the prediction accuracy rate for them is 97%，and in complete dataset,it's only 50%，compared with before, there has been a big improvement, so the prediction results are credible.

## 4.5 Survival prediction<a id="16"></a>

In [1]:
combine[(combine.Pclass_Gender.isin(["P1-Man","P3-Man"]))
        &(combine.Survived.isna())&(combine.If_Survived==1)][["Pclass_Gender","Group_size","Ave_Survival","Ticket"]]

- Except that [931] has a high probability of surviving, the probability of the other three passengers surviving is unclear.Therefore, in addition to [931], the other three passengers will be predicted to die based on the probability in the complete dataset.

In [1]:
combine.loc[(combine.If_Survived==0),"Predict"] = 0
combine.loc[(combine.If_Survived==1),"Predict"] = 1
combine.loc[(combine.PassengerId.isin([1036,1047,1210])),"Predict"] = 0

# 5.Analysis of the remaining datasets<a id="17"></a>

## 5.1 Learn about the remaining training data and test data<a id="18"></a>

In [1]:
combine = combine[["PassengerId","Age","Embarked","Survived","Pclass_Gender","Fare_PP","Cabin_Title","Predict"]]

In [1]:
remain_train = combine[(combine.Predict.isna())&(combine.Survived.notna())]
remain_test = combine[(combine.Predict.isna())&(combine.Survived.isna())]

In [1]:
print("Pclass_Gender distribution in train data:",remain_train.Pclass_Gender.value_counts())
print("-"*50)
print("Pclass_Gender distribution in test data:",remain_test.Pclass_Gender.value_counts())

In [1]:
#P1-Man
survival_in_remain = remain_train[(remain_train.Pclass_Gender=="P1-Man")]["Survived"].mean()
survival_in_all = combine[(combine.Pclass_Gender=="P1-Man")]["Survived"].mean()
print(round(survival_in_all,2),round(survival_in_remain,2))

In [1]:
#P3-Man
survival_in_remain = remain_train[(remain_train.Pclass_Gender=="P3-Man")]["Survived"].mean()
survival_in_all = combine[(combine.Pclass_Gender=="P3-Man")]["Survived"].mean()
print(round(survival_in_all,2),round(survival_in_remain,2))

In [1]:
#P3-Woman
survival_in_remain =remain_train[(remain_train.Pclass_Gender=="P3-Woman")]["Survived"].mean()
survival_in_all = combine[(combine.Pclass_Gender=="P3-Woman")]["Survived"].mean()
print(round(survival_in_all,2),round(survival_in_remain,2))

**Summary:**
- Since the survival rates of "P1-Man" and "P3-Man" are consistent in the complete dataset and the remaining dataset, the relationship between features and "Survived" of these passengers will be observed in the complete dataset.
- the relationship between features and "Survived" of "P3-Woman" will be observed in the remaining dataset.
- The survivability of "P3-Boy" will be predicted to be died based on its survival rate in the complete dataset.

## 5.2 Visualization of the relationship between passenger's features and survival rate<a id="19"></a>

In [1]:
%%HTML
<div class='tableauPlaceholder' id='viz1563451480836' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;An&#47;AnOverviewofWoman2&#47;AnOverviewofP3-Woman&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AnOverviewofWoman2&#47;AnOverviewofP3-Woman' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;An&#47;AnOverviewofWoman2&#47;AnOverviewofP3-Woman&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1563451480836');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

**Observation conclusion：**
- 1.When considering the combined effects of all features, the average survival rate of "P3-Woman" was 65%.
- 2.When considering the effect of each feature on "Survived" alone, only when age is between 36 and 47,the survival rate is 0%,However, the sample size of it is too low and the prediction error is large.Therefore, for "P3-Woman",they will all be predicted to survive based on the average survival rate.

In [1]:
%%HTML
<div class='tableauPlaceholder' id='viz1563448568883' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;An&#47;AnOverviewofP1-Man&#47;AnOverviewofP1-Man&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AnOverviewofP1-Man&#47;AnOverviewofP1-Man' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;An&#47;AnOverviewofP1-Man&#47;AnOverviewofP1-Man&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1563448568883');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

**Observation conclusion：**
- 1.When considering the combined effects of all features, the average survival rate of "P1-Man" was 35%.
- 2.When considering the effect of each feature on "Survived" alone, there are no categories of survival rates above mortality,Therefore, for "P1-Man",they will all be predicted to died based on the average survival rate.

In [1]:
%%HTML
<div class='tableauPlaceholder' id='viz1563451555664' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;An&#47;AnOverviewofP3-Man&#47;AnOverviewofP3-Man&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AnOverviewofP3-Man&#47;AnOverviewofP3-Man' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;An&#47;AnOverviewofP3-Man&#47;AnOverviewofP3-Man&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1563451555664');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

**Observation conclusion：**
- 1.When considering the combined effects of all features, the average survival rate of "P3-Man" was only 11%.
- 2.When considering the effect of each feature on "Survived" alone, there are no categories of survival rates above mortality,Therefore, for "P3-Man",they will all be predicted to died based on the average survival rate.

## 5.3 Survival prediction and result submission<a id="20"></a>

In [1]:
combine.loc[(combine.Predict.isna())&(combine.Survived.isna())&(combine.Pclass_Gender=="P3-Woman"),"Predict"] = 1
combine.loc[(combine.Predict.isna())&(combine.Survived.isna()),"Predict"]=0

In [1]:
final_submit = pd.DataFrame({"PassengerId":combine[combine.Survived.isna()].PassengerId.values,
                       "Survived":combine[combine.Survived.isna()].Predict.values})
final_submit["Survived"] = final_submit.Survived.astype("int64")
final_submit.tail(2)

In [1]:
final_submit.to_csv("Titanic.csv",index = False)